In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/')
import warnings
warnings.filterwarnings('ignore')



import pandas as pd
import numpy as np
import seaborn as sns


import sklearn
from sklearn import preprocessing
from sklearn import metrics
from sklearn import model_selection


#import catboost as cb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open("/content/drive/MyDrive/날씨/lgb_train.pkl","rb") as fr:
    data = pickle.load(fr)
train = data.copy()

In [ ]:
df_2005  =pd.read_csv('/content/drive/MyDrive/날씨/data_자외선/원본(오류수정)/uv_2005.csv')

In [ ]:
df_2005.loc[df_2005['202005_uv.stn']==13,:]

,Unnamed: 0,202005_uv.yyyymmdd,202005_uv.hhnn,202005_uv.stn,202005_uv.lon,202005_uv.lat,202005_uv.uv,202005_uv.band1,202005_uv.band2,202005_uv.band3,...,202005_uv.band12,202005_uv.band13,202005_uv.band14,202005_uv.band15,202005_uv.band16,202005_uv.solarza,202005_uv.sateza,202005_uv.esr,202005_uv.height,202005_uv.landtype
0,1,20200501,0,13,126.16,33.3,0.0,0.00000,-0.00057,-0.00030,...,252.91737,277.63245,277.03649,275.67037,262.71147,130.35092,38.96359,-6.11462,71.0,0
15,16,20200501,10,13,126.16,33.3,0.0,-0.00057,-0.00057,0.00060,...,254.92374,281.76290,281.86880,280.33071,266.54655,130.77681,38.96359,-6.11462,71.0,0
30,31,20200501,20,13,126.16,33.3,0.0,-999.00000,-999.00000,-999.00000,...,-999.00000,-999.00000,-999.00000,-999.00000,-999.00000,131.08983,38.96359,-6.11462,71.0,0
45,46,20200501,30,13,126.16,33.3,0.0,-0.00170,-0.00057,-0.00745,...,256.77138,285.37234,285.75315,284.55141,269.92091,131.28775,38.96359,-6.11462,71.0,0
60,61,20200501,40,13,126.16,33.3,0.0,0.00000,-0.00057,-0.00030,...,256.65245,285.01048,285.49655,283.95280,269.52565,131.33348,38.96359,-6.11462,71.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66885,66886,20200531,2310,13,126.16,33.3,0.0,0.00000,0.00000,0.00060,...,246.69440,261.56910,260.98100,259.65300,252.17720,120.26510,38.96359,-4.59716,71.0,0
66900,66901,20200531,2320,13,126.16,33.3,0.0,0.00000,-0.00057,0.00000,...,253.08520,272.66780,272.27180,270.49790,260.28160,121.10720,38.96359,-4.59716,71.0,0
66915,66916,20200531,2330,13,126.16,33.3,0.0,0.00000,-0.00057,0.00089,...,252.17780,271.26770,270.73520,268.92420,259.04690,121.86390,38.96359,-4.59716,71.0,0
66930,66931,20200531,2340,13,126.16,33.3,0.0,0.00000,0.00000,-0.00030,...,242.38520,253.69760,252.60850,251.09570,245.12430,122.53170,38.96359,-4.59716,71.0,0


In [ ]:
import glob
forecast = glob.glob('/content/drive/MyDrive/날씨/형한결/forecast/*.csv')

In [ ]:
forecast

['/content/drive/MyDrive/날씨/형한결/forecast/2020.csv',
 '/content/drive/MyDrive/날씨/형한결/forecast/2021.csv',
 '/content/drive/MyDrive/날씨/형한결/forecast/2022.csv',
 '/content/drive/MyDrive/날씨/형한결/forecast/기상현상(train).csv',
 '/content/drive/MyDrive/날씨/형한결/forecast/기상현상(test).csv']

In [ ]:
df_list = []
for i in forecast[0:3]:
  df_list.append(pd.read_csv(i,encoding='cp949'))
  

In [ ]:
forecast_df  =  pd.concat(df_list)

In [ ]:
set(forecast_df['지점'])

{105, 108, 112, 115, 131, 133, 138, 143, 146, 152, 156, 159, 165, 185, 235}

In [ ]:
set(train.stn)

{13, 105, 108, 112, 115, 131, 132, 133, 138, 143, 146, 152, 156, 159, 165}

In [ ]:
forecast_df.isna().sum()

지점                   0
지점명                  0
일시                   0
기온(°C)             113
기온 QC플래그        273734
강수량(mm)         243645
강수량 QC플래그       220712
풍속(m/s)            106
풍속 QC플래그        273905
습도(%)               33
습도 QC플래그        273914
증기압(hPa)            68
이슬점온도(°C)           95
현지기압(hPa)          194
현지기압 QC플래그      273751
일조(hr)          123592
일조 QC플래그        150420
일사(MJ/m2)       143713
일사 QC플래그        130295
전운량(10분위)         1105
중하층운량(10분위)        883
운형(운형약어)        145650
최저운고(100m )     121664
시정(10m)            200
지면상태(지면상태코드)    273960
현상번호(국내식)       211983
지면온도(°C)           127
지면온도 QC플래그      272321
dtype: int64

- 강수량
- 운형
- 일조
- 일사
- 운형
- 최저 운고
- 지면 상태
- 현상번호


In [ ]:
pp_fc = forecast_df.copy()
for col in list(pp_fc.columns):
  if 'QC플래그' in col:
    pp_fc.drop(col,axis=1,inplace=True)

In [ ]:
pp_fc.columns

Index(['지점', '지점명', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '증기압(hPa)',
       '이슬점온도(°C)', '현지기압(hPa)', '일조(hr)', '일사(MJ/m2)', '전운량(10분위)',
       '중하층운량(10분위)', '운형(운형약어)', '최저운고(100m )', '시정(10m)', '지면상태(지면상태코드)',
       '현상번호(국내식)', '지면온도(°C)'],
      dtype='object')

In [ ]:
pp_fc.drop('지면상태(지면상태코드)',axis=1,inplace=True)

In [ ]:
pp_fc.isna().sum()

지점                  0
지점명                 0
일시                  0
기온(°C)            113
강수량(mm)        243645
풍속(m/s)           106
습도(%)              33
증기압(hPa)           68
이슬점온도(°C)          95
현지기압(hPa)         194
일조(hr)         123592
일사(MJ/m2)      143713
전운량(10분위)        1105
중하층운량(10분위)       883
운형(운형약어)       145650
최저운고(100m )    121664
시정(10m)           200
현상번호(국내식)      211983
지면온도(°C)          127
dtype: int64

In [ ]:
type(list(pp_fc['강수량(mm)'][1])[0])

float

In [ ]:
#@title 기본 제목 텍스트
pp_fc.loc[pp_fc['강수량(mm)'].isna(),['강수량(mm)','일시']]

,강수량(mm),일시
0,NaN,2020-01-01 00:00
1,NaN,2020-01-01 01:00
2,NaN,2020-01-01 02:00
3,NaN,2020-01-01 03:00
4,NaN,2020-01-01 04:00
...,...,...
10795,NaN,2022-06-30 19:00
10796,NaN,2022-06-30 20:00
10797,NaN,2022-06-30 21:00
10798,NaN,2022-06-30 22:00


In [ ]:
pp_fc.drop('강수량(mm)',axis=1,inplace=True)
pp_fc.drop('지점명',axis=1,inplace=True)
pp_fc.drop(['일조(hr)','일사(MJ/m2)'],axis=1,inplace=True)
pp_fc.drop(['최저운고(100m )'],axis=1,inplace=True)

In [ ]:
pp_fc.isna().sum()

지점                  0
일시                  0
기온(°C)            113
풍속(m/s)           106
습도(%)              33
증기압(hPa)           68
이슬점온도(°C)          95
현지기압(hPa)         194
전운량(10분위)        1105
중하층운량(10분위)       883
운형(운형약어)       145650
시정(10m)           200
현상번호(국내식)      211983
지면온도(°C)          127
dtype: int64

In [ ]:
set(pp_fc['운형(운형약어)'].dropna())

{'Ac',
 'AcCc',
 'AcCcCi',
 'AcCi',
 'AcCs',
 'AcCsCi',
 'As',
 'AsAc',
 'AsAcCi',
 'AsCi',
 'AsCs',
 'CbAcCi',
 'CbCu',
 'CbCuAs',
 'CbCuNs',
 'CbCuScCi',
 'CbCuScNs',
 'CbNs',
 'CbSc',
 'CbScAc',
 'CbScAcCi',
 'CbScAs',
 'CbScCi',
 'CbScCs',
 'CbScNs',
 'CbSt',
 'CbStAs',
 'CbStCi',
 'CbStNs',
 'Cc',
 'CcCi',
 'Ci',
 'Cs',
 'CsCc',
 'CsCi',
 'Cu',
 'CuAc',
 'CuAcCc',
 'CuAcCi',
 'CuAs',
 'CuCc',
 'CuCcCi',
 'CuCi',
 'CuCs',
 'CuCsCi',
 'CuNs',
 'CuSc',
 'CuScAc',
 'CuScAcCi',
 'CuScAcCs',
 'CuScAs',
 'CuScAsCi',
 'CuScCc',
 'CuScCcCi',
 'CuScCi',
 'CuScCs',
 'CuScCsCi',
 'CuScNs',
 'CuSt',
 'CuStNs',
 'Ns',
 'Sc',
 'ScAc',
 'ScAcCc',
 'ScAcCcCi',
 'ScAcCi',
 'ScAcCs',
 'ScAs',
 'ScAsCi',
 'ScCc',
 'ScCcCi',
 'ScCi',
 'ScCs',
 'ScCsCc',
 'ScCsCi',
 'ScNs',
 'St',
 'StAc',
 'StAcCi',
 'StAs',
 'StAsCi',
 'StCcCi',
 'StCi',
 'StCs',
 'StNs'}

In [ ]:
pp_fc['현상번호(국내식)'].isna().sum()

211983

In [ ]:
pp_fc['temperature'] = pp_fc['기온(°C)'].interpolate(methodstr='cubic')
pp_fc['humidity'] = pp_fc['습도(%)'].interpolate(methodstr='cubic')
pp_fc['wind_speed'] = pp_fc['풍속(m/s)'].interpolate(methodstr='cubic')
pp_fc['steam_pressure'] = pp_fc['증기압(hPa)'].interpolate(methodstr='cubic')
pp_fc['dew'] = pp_fc['이슬점온도(°C)'].interpolate(methodstr='cubic')
pp_fc['pressure'] = pp_fc['현지기압(hPa)'].interpolate(methodstr='cubic')
pp_fc['pressure'] = pp_fc['pressure'].bfill()
pp_fc['visibility'] = pp_fc['시정(10m)'].interpolate(methodstr='cubic')
pp_fc['temperature'] = pp_fc['지면온도(°C)'].interpolate(methodstr='cubic')
pp_fc.drop(['기온(°C)','습도(%)','풍속(m/s)','증기압(hPa)','이슬점온도(°C)','현지기압(hPa)','시정(10m)','지면온도(°C)'],axis=1,inplace=True)
pp_fc['date_time'] = pp_fc['일시']
pp_fc['date_time'] = pd.to_datetime(pp_fc['date_time'])
pp_fc['stn'] = pp_fc['지점']
pp_fc['total_cloud'] = pp_fc['전운량(10분위)']
pp_fc['mid_cloud'] = pp_fc['중하층운량(10분위)']
pp_fc.drop(['지점','전운량(10분위)','중하층운량(10분위)'],axis=1,inplace=True)
pp_fc['observe_num'] = pp_fc['현상번호(국내식)']
pp_fc['cloud_shape'] = pp_fc['운형(운형약어)']
pp_fc.drop(['일시','운형(운형약어)','현상번호(국내식)'],axis=1,inplace=True)
pp_fc['stn'] = pp_fc['stn'].replace(185,13)
pp_fc['stn'] = pp_fc['stn'].replace(235,132)

In [ ]:
train['date_time'] = train.index.date

In [ ]:
train['date_time']

date_time
2020-01-01 00:00:00    2020-01-01
2020-01-01 00:10:00    2020-01-01
2020-01-01 00:20:00    2020-01-01
2020-01-01 00:30:00    2020-01-01
2020-01-01 00:40:00    2020-01-01
                          ...    
2021-12-31 23:10:00    2021-12-31
2021-12-31 23:20:00    2021-12-31
2021-12-31 23:30:00    2021-12-31
2021-12-31 23:40:00    2021-12-31
2021-12-31 23:50:00    2021-12-31
Name: date_time, Length: 1578960, dtype: object

In [ ]:
pp_fc['date_time'] = pp_fc['date_time'].dt.date

In [ ]:
train.head()

,uv,band1,band2,band3,band4,band5,band6,band7,band8,band9,...,Month sin,Month cos,Day sin,Day cos,2/Day sin,2/Day cos,3/Day sin,3/Day cos,stn,date_time
date_time,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,0.0,0.009771,0.009774,0.011324,0.010648,0.019778,0.018657,0.536633,0.680527,0.747787,...,0.487099,0.999834,0.500000,1.000000,0.500000,1.000000,0.500000,1.000000,131,2020-01-01
2020-01-01 00:10:00,0.0,0.009279,0.009774,0.010194,0.011053,0.018288,0.017717,0.534080,0.677492,0.745142,...,0.487816,0.999852,0.521810,0.999524,0.543578,0.998097,0.565263,0.995722,131,2020-01-01
2020-01-01 00:20:00,0.0,0.009279,0.009286,0.011324,0.010378,0.019778,0.018657,0.535360,0.675969,0.743373,...,0.488532,0.999868,0.543578,0.998097,0.586824,0.992404,0.629410,0.982963,131,2020-01-01
2020-01-01 00:30:00,0.0,0.009279,0.009286,0.012740,0.010243,0.020269,0.018657,0.513766,0.679011,0.743816,...,0.489249,0.999884,0.565263,0.995722,0.629410,0.982963,0.691342,0.961940,131,2020-01-01
2020-01-01 00:40:00,0.0,0.008796,0.009286,0.011324,0.010648,0.019287,0.017717,0.509430,0.679011,0.744700,...,0.489966,0.999899,0.586824,0.992404,0.671010,0.969846,0.750000,0.933013,131,2020-01-01


In [ ]:
pp_fc.head()

,temperature,humidity,wind_speed,steam_pressure,dew,pressure,visibility,date_time,stn,total_cloud,mid_cloud,observe_num,cloud_shape
0,-4.1,24.0,4.3,1.2,-20.7,1021.7,5000.0,2020-01-01,105,0.0,0.0,NaN,NaN
1,-3.6,37.0,5.2,1.9,-15.0,1021.7,5000.0,2020-01-01,105,0.0,0.0,NaN,NaN
2,-3.5,38.0,3.9,2.1,-14.0,1021.7,5000.0,2020-01-01,105,0.0,0.0,NaN,NaN
3,-3.2,45.0,5.6,2.5,-11.8,1021.7,2816.0,2020-01-01,105,0.0,0.0,NaN,NaN
4,-2.8,46.0,5.1,2.6,-11.0,1021.7,3102.0,2020-01-01,105,0.0,0.0,NaN,NaN


In [ ]:
set(pp_fc['stn']) == set(train['stn'])

True

In [ ]:
train['date_time'] = train.index
pp_fc = pp_fc.rename(columns={'date_time':'datetime'})
train = train.rename(columns={'date_time':'datetime'})

In [ ]:
#pd.merge(pp_fc,train,on=['datetime','stn'])